In [46]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [32]:
gender_submission = pd.read_csv('data/gender_submission.csv', sep = ',')
test = pd.read_csv('data/test.csv', sep = ',')
train = pd.read_csv('data/train.csv', sep = ',')

train = pd.get_dummies(train)
test = pd.get_dummies(test)

train_test, train_valid = train_test_split(train, test_size=0.2, random_state=42)

In [ ]:
"""
pred_survived = train_test.groupby('Pclass')['Survived'].agg(lambda p: p.value_counts().idxmax())
valid_df['Predicted'] = valid_df['Pclass'].map(pred_survived)
accuracy = accuracy_score(valid_df['Survived'], valid_df['Predicted'])
print("Accuracy: ", accuracy)

pd_data = pd.merge(gender_submission, test_data)
pd_data['Predicted'] = pd_data['Pclass'].map(pred_survived)
output = pd.DataFrame({'PassengerId': pd_data.PassengerId, 'Survived': pd_data['Survived']})
output.to_csv('my_submission.csv', index=False)

print("Your submission was successfully saved!")
accuracy = accuracy_score(pd_data['Survived'], pd_data['Predicted'])
print(accuracy)
"""

In [33]:
features = ['Pclass']
x = train_test[features]
y = train_test["Survived"]

model = LogisticRegression()
model.fit(x, y)
predictions = model.predict(train_valid[features])

accuracy = accuracy_score(train_valid["Survived"], predictions)
print(accuracy)

0.7039106145251397


In [41]:
features = ['Sex_female', 'Sex_male', 'Age']

test.fillna({"Age": train["Age"].median(skipna=True)}, inplace=True)
train.fillna({"Age": train["Age"].median(skipna=True)}, inplace=True)

x = train[features]
y = train["Survived"]

model = LogisticRegression()
model.fit(x, y)
predictions = model.predict(test[features])

accuracy = accuracy_score(gender_submission["Survived"], predictions)
print(accuracy)

1.0


In [93]:
features = ['Sex_female', 'Sex_male', 'Pclass']

test.fillna({"Age": train["Age"].median(skipna=True)}, inplace=True)
train.fillna({"Age": train["Age"].median(skipna=True)}, inplace=True)

x = train[features]
y = train["Survived"]

model = RandomForestClassifier(n_estimators=1000, max_depth=5, random_state=1)
model.fit(x, y)
predictions = model.predict(test[features])

accuracy = accuracy_score(gender_submission["Survived"], predictions)
print(accuracy)

1.0
